In [6]:
import pandas as pd

In [7]:
pd.read_csv("log/describe_all.csv")

,global_size,nx,nvx,kernel,error_mean,error_std,runtime_mean,runtime_std,cellspersec_mean,cellspersec_std,throughput_mean,throughput_std,gpu
0,10000,100,100,BasicRange2D,0.004607,9.142795e-19,0.712581,0.315975,0.062798,0.018474,1.004775,0.295578,0
1,40000,200,200,BasicRange2D,0.000282,5.714247e-20,2.158293,0.475841,0.076616,0.012614,1.225852,0.201819,0
2,160000,400,400,BasicRange2D,0.000017,0.000000e+00,7.545414,0.699403,0.085371,0.006621,1.365934,0.105936,0
3,10000,100,100,BasicRange1D,0.004607,9.142795e-19,0.511812,0.102864,0.080661,0.014000,1.290572,0.223998,0
4,40000,200,200,BasicRange1D,0.000282,5.714247e-20,1.575075,0.220085,0.102976,0.011133,1.647618,0.178130,0
5,160000,400,400,BasicRange1D,0.000017,0.000000e+00,5.832999,0.163341,0.109797,0.003051,1.756760,0.048816,0
6,10000,100,100,Hierarchical,0.004607,9.142795e-19,0.477050,0.068880,0.085400,0.012001,1.366392,0.192013,0
7,40000,200,200,Hierarchical,0.000282,5.714247e-20,1.479052,0.155940,0.109109,0.009872,1.745750,0.157955,0
8,160000,400,400,Hierarchical,0.000017,0.000000e+00,5.495331,0.122120,0.116512,0.002497,1.864199,0.039952,0
9,10000,100,100,Scoped,0.004607,9.142795e-19,0.436416,0.081033,0.094258,0.015675,1.508133,0.250803,0
